## 数据读取

In [ ]:
root_path="testset"
train_path="D:\Download\Dataset\Dataset\\train.txt"
import pandas as pd
train_data = pd.read_csv("D:\Download\Dataset\Dataset\\train.txt",names=["label","comment"],sep="\t")
print(train_data.head())